In [2]:
import json
from sqlalchemy import create_engine, String, Integer
from sqlalchemy.sql import text, bindparam

In [11]:
import pandas as pd
import geopandas as gpd

In [39]:
with open("pg-credentials.json", "r") as f_in:
    pg_creds = json.load(f_in)

# mapbox
with open("mapbox_token.json", "r") as mb_token:
    MAPBOX_TOKEN = json.load(mb_token)["token"]

# load credentials from JSON file
HOST = pg_creds["HOST"]
USERNAME = pg_creds["USERNAME"]
PASSWORD = pg_creds["PASSWORD"]
DATABASE = pg_creds["DATABASE"]
PORT = pg_creds["PORT"]
engine = create_engine(f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [49]:
def bus5(lng,lat):
    query = text("""
    SELECT "Stop_Name","Mode","Latitude","Longitude",geometry,
        ROUND(ST_Distance(geometry::geography,ST_MakePoint(-75.1927,39.9522)::geography))AS Distance
    FROM stops_philly
    WHERE "Mode"!='Highspeed'
    ORDER BY Distance
    LIMIT 5
""")
    #gdf=[dict(row) for row in engine.execute(query, longitude=lng, latitude=lat).fetchall()]
    gdf=gpd.GeoDataFrame([dict(row) for row in engine.execute(query, longitude=lng, latitude=lat).fetchall()])
    return gdf

In [41]:
def rail2(lng,lat):
    query = text("""
    SELECT "Stop_Name","Mode","Latitude","Longitude",geometry,
        ST_Distance(geometry::geography,ST_MakePoint(-75.1927,39.9522)::geography)AS Distance
    FROM stops_philly
    WHERE "Mode"='Highspeed'
    ORDER BY Distance
    LIMIT 2
""")
    #gdf=[dict(row) for row in engine.execute(query, longitude=lng, latitude=lat).fetchall()]
    gdf=gpd.GeoDataFrame([dict(row) for row in engine.execute(query, longitude=lng, latitude=lat).fetchall()])
    return gdf

In [55]:
bus=bus5(-75.1927,39.9522)

In [57]:
bus.dtypes

Stop_Name     object
Mode          object
Latitude     float64
Longitude    float64
geometry      object
distance     float64
dtype: object

In [31]:
type(tmp.geometry)

pandas.core.series.Series